---
title: Clean the main information CSV file scraped before
---
To save the space and make my code more manageable,
I put the convenience functions in a module and import them.

In [1]:
using DataFrames, Chain
using CSV, JLD2
using Printf
using Dates
using Crowdfunding: parse_datetime, convert_time_zone, compute_time_index
using Crowdfunding: parse_price_quantity_string, compute_avg_price
using Crowdfunding: complete, interpolate_missing_linear, fill_missing
project_dir = dirname(@__DIR__)

"d:\\work\\crowdfunding"

In [2]:
main_info_df = CSV.read("$project_dir/data/prev_data/merged_main_info_df.csv",DataFrame)
describe(main_info_df)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,Type
1,project_id,1.2259e5,37494,122962.0,123926,0,Int64
2,start_time,,2022-08-10 19:00:00,,2022-11-24 20:00:00,7,"Union{Missing, String31}"
3,end_time,,2022-10-10 15:00:00,,2023-01-17 12:00:00,7,"Union{Missing, String31}"
4,creator_id,7.24447e8,924604.0,1.00258e9,1.00438e9,7,"Union{Missing, Float64}"
5,category,,出版,,食品,7,"Union{Missing, String15}"
6,price_quantity,,"[(1.0, 0), (2.0, 1), (4.0, 1), (5.0, 4), (18.0, 93), (36.0, 47), (36.0, 25), (48.0, 25), (68.0, 252), (78.0, 66), (130.0, 99), (136.0, 13), (258.0, 81), (268.0, 2)]",,"[(999.0, 9), (999.0, 9), (999.0, 3), (519.0, 28), (519.0, 13), (519.0, 15), (129.0, 24), (129.0, 33), (129.0, 33), (129.0, 10), (95.0, 9), (95.0, 7), (95.0, 16), (95.0, 7), (95.0, 3), (95.0, 3), (139.0, 0), (139.0, 9), (139.0, 1), (139.0, 0), (99.0, 0), (99.0, 0), (99.0, 0), (99.0, 0), (99.0, 0), (99.0, 0), (549.0, 4), (549.0, 0)]",7,"Union{Missing, String}"
7,goal,27739.9,500.0,10000.0,1.5e6,7,"Union{Missing, Float64}"
8,update_num,4.04705,0.0,3.0,47.0,7,"Union{Missing, Float64}"
9,attention,884.297,0.0,411.0,20191.0,7,"Union{Missing, Float64}"


## Check missing values

In [3]:
subset(main_info_df,All()=>ByRow((x...)->any(ismissing,x)))

Row,project_id,start_time,end_time,creator_id,category,price_quantity,goal,update_num,attention,comment_num,pledged,backer_num,scraped_time,status
,Int64,String31?,String31?,Float64?,String15?,String?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,String31,String7
1,122554,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,2022-10-24 22:20:12.349268,active
2,122554,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,2022-10-25 22:21:43.428987,active
3,122554,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,2022-10-24 10:24:08.745026,active
4,122560,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,2022-10-10 11:14:44,active
5,122857,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,2022-10-10 11:14:44,active
6,122704,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,2022-10-10 11:14:44,active
7,122554,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,2022-10-25 10:18:21.394337,active


These missing values are due to the bugs of the crowdfunding websites.
Just drop them.

In [4]:
dropmissing!(main_info_df)

Row,project_id,start_time,end_time,creator_id,category,price_quantity,goal,update_num,attention,comment_num,pledged,backer_num,scraped_time,status
,Int64,String31,String31,Float64,String15,String,Float64,Float64,Float64,Float64,Float64,Float64,String31,String7
1,122725,2022-10-12 18:00:00,2022-11-01 19:00:00,1.00172e9,桌游,"[(758.0, 9), (1088.0, 420), (198.0, 123), (168.0, 57), (25.0, 4), (45.0, 249)]",100000.0,16.0,4164.0,597.0,509117.0,423.0,2022-10-22 10:21:21.443065,active
2,122816,2022-09-29 19:00:00,2022-11-22 23:59:00,1.09839e6,出版,"[(119.0, 496), (129.0, 12), (189.0, 30), (328.0, 18), (388.0, 60)]",50000.0,2.0,1059.0,99.0,95207.0,584.0,2022-10-22 10:21:21.443065,active
3,122810,2022-09-21 19:00:00,2022-11-01 18:16:00,1.19279e6,出版,"[(678.0, 198), (698.0, 122), (249.0, 329), (478.0, 130), (88.0, 62), (500.0, 0)]",50000.0,11.0,1624.0,257.0,3.69063e5,714.0,2022-10-22 10:21:21.443065,active
4,120778,2022-09-25 16:00:00,2022-11-06 12:00:00,1.00366e9,科技,"[(79.0, 30), (89.0, 10), (169.0, 3), (239.0, 22), (375.0, 0), (99.0, 12)]",20000.0,1.0,326.0,99.0,10213.0,53.0,2022-10-22 10:21:21.443065,active
5,122864,2022-10-08 18:00:00,2022-11-08 18:00:00,1.01189e6,桌游,"[(38.0, 65), (38.0, 10), (78.0, 1), (88.0, 0), (98.0, 46), (98.0, 0), (98.0, 0), (108.0, 10), (108.0, 1), (128.0, 0), (128.0, 0), (128.0, 0), (148.0, 0), (168.0, 0), (498.0, 0), (718.0, 0)]",3000.0,2.0,617.0,47.0,8646.88,126.0,2022-10-22 10:21:21.443065,active
6,122473,2022-09-30 12:00:00,2022-11-27 12:00:00,1.00372e9,设计,"[(39.0, 0), (39.0, 1), (79.0, 9)]",2800.0,2.0,98.0,13.0,670.0,10.0,2022-10-22 10:21:21.443065,active
7,122618,2022-09-23 20:00:00,2022-11-20 23:59:00,1.00345e9,设计,"[(158.0, 37), (175.0, 2), (168.0, 10), (185.0, 0), (399.0, 3), (435.0, 2), (188.0, 21), (205.0, 0), (198.0, 34), (215.0, 3), (429.0, 14), (465.0, 3), (725.0, 2), (785.0, 5)]",30000.0,3.0,1891.0,63.0,34031.0,118.0,2022-10-22 10:21:21.443065,active
8,123141,2022-10-17 19:00:00,2022-11-11 22:00:00,1.00353e9,出版,"[(28.0, 7), (58.0, 60), (88.0, 80), (98.0, 197), (128.0, 7), (198.0, 13), (376.0, 80), (386.0, 32)]",40000.0,5.0,754.0,78.0,74507.3,299.0,2022-10-22 10:21:21.443065,active
9,123148,2022-10-14 19:00:00,2022-11-14 21:00:00,1.0031e9,设计,"[(288.0, 50), (288.0, 50), (298.0, 9), (298.0, 35)]",1000.0,2.0,668.0,33.0,41922.0,131.0,2022-10-22 10:21:21.443065,active


## Convert column types

In [5]:
Base.show(io::IO, f::Float64) = @printf(io, "%.0f", f) # ids are misinterpreted as floats
transform!(main_info_df,
[:project_id,:creator_id].=>x->repr.(x),
:status=>x->x.=="active";
renamecols=false)

Row,project_id,start_time,end_time,creator_id,category,price_quantity,goal,update_num,attention,comment_num,pledged,backer_num,scraped_time,status
,String,String31,String31,String,String15,String,Float64,Float64,Float64,Float64,Float64,Float64,String31,Bool
1,122725,2022-10-12 18:00:00,2022-11-01 19:00:00,1001721653,桌游,"[(758.0, 9), (1088.0, 420), (198.0, 123), (168.0, 57), (25.0, 4), (45.0, 249)]",100000,16,4164,597,509117,423,2022-10-22 10:21:21.443065,true
2,122816,2022-09-29 19:00:00,2022-11-22 23:59:00,1098393,出版,"[(119.0, 496), (129.0, 12), (189.0, 30), (328.0, 18), (388.0, 60)]",50000,2,1059,99,95207,584,2022-10-22 10:21:21.443065,true
3,122810,2022-09-21 19:00:00,2022-11-01 18:16:00,1192788,出版,"[(678.0, 198), (698.0, 122), (249.0, 329), (478.0, 130), (88.0, 62), (500.0, 0)]",50000,11,1624,257,369063,714,2022-10-22 10:21:21.443065,true
4,120778,2022-09-25 16:00:00,2022-11-06 12:00:00,1003655938,科技,"[(79.0, 30), (89.0, 10), (169.0, 3), (239.0, 22), (375.0, 0), (99.0, 12)]",20000,1,326,99,10213,53,2022-10-22 10:21:21.443065,true
5,122864,2022-10-08 18:00:00,2022-11-08 18:00:00,1011894,桌游,"[(38.0, 65), (38.0, 10), (78.0, 1), (88.0, 0), (98.0, 46), (98.0, 0), (98.0, 0), (108.0, 10), (108.0, 1), (128.0, 0), (128.0, 0), (128.0, 0), (148.0, 0), (168.0, 0), (498.0, 0), (718.0, 0)]",3000,2,617,47,8647,126,2022-10-22 10:21:21.443065,true
6,122473,2022-09-30 12:00:00,2022-11-27 12:00:00,1003718051,设计,"[(39.0, 0), (39.0, 1), (79.0, 9)]",2800,2,98,13,670,10,2022-10-22 10:21:21.443065,true
7,122618,2022-09-23 20:00:00,2022-11-20 23:59:00,1003452041,设计,"[(158.0, 37), (175.0, 2), (168.0, 10), (185.0, 0), (399.0, 3), (435.0, 2), (188.0, 21), (205.0, 0), (198.0, 34), (215.0, 3), (429.0, 14), (465.0, 3), (725.0, 2), (785.0, 5)]",30000,3,1891,63,34031,118,2022-10-22 10:21:21.443065,true
8,123141,2022-10-17 19:00:00,2022-11-11 22:00:00,1003527584,出版,"[(28.0, 7), (58.0, 60), (88.0, 80), (98.0, 197), (128.0, 7), (198.0, 13), (376.0, 80), (386.0, 32)]",40000,5,754,78,74507,299,2022-10-22 10:21:21.443065,true
9,123148,2022-10-14 19:00:00,2022-11-14 21:00:00,1003103318,设计,"[(288.0, 50), (288.0, 50), (298.0, 9), (298.0, 35)]",1000,2,668,33,41922,131,2022-10-22 10:21:21.443065,true


convert time zone

In [6]:
transform!(main_info_df,
[:start_time,:end_time].=>ByRow(parse_datetime),
:scraped_time=>ByRow(x->parse_datetime(x)|>convert_time_zone);renamecols=false)

Row,project_id,start_time,end_time,creator_id,category,price_quantity,goal,update_num,attention,comment_num,pledged,backer_num,scraped_time,status
,String,DateTime,DateTime,String,String15,String,Float64,Float64,Float64,Float64,Float64,Float64,DateTime,Bool
1,122725,2022-10-12T18:00:00,2022-11-01T19:00:00,1001721653,桌游,"[(758.0, 9), (1088.0, 420), (198.0, 123), (168.0, 57), (25.0, 4), (45.0, 249)]",100000,16,4164,597,509117,423,2022-10-23T01:21:21,true
2,122816,2022-09-29T19:00:00,2022-11-22T23:59:00,1098393,出版,"[(119.0, 496), (129.0, 12), (189.0, 30), (328.0, 18), (388.0, 60)]",50000,2,1059,99,95207,584,2022-10-23T01:21:21,true
3,122810,2022-09-21T19:00:00,2022-11-01T18:16:00,1192788,出版,"[(678.0, 198), (698.0, 122), (249.0, 329), (478.0, 130), (88.0, 62), (500.0, 0)]",50000,11,1624,257,369063,714,2022-10-23T01:21:21,true
4,120778,2022-09-25T16:00:00,2022-11-06T12:00:00,1003655938,科技,"[(79.0, 30), (89.0, 10), (169.0, 3), (239.0, 22), (375.0, 0), (99.0, 12)]",20000,1,326,99,10213,53,2022-10-23T01:21:21,true
5,122864,2022-10-08T18:00:00,2022-11-08T18:00:00,1011894,桌游,"[(38.0, 65), (38.0, 10), (78.0, 1), (88.0, 0), (98.0, 46), (98.0, 0), (98.0, 0), (108.0, 10), (108.0, 1), (128.0, 0), (128.0, 0), (128.0, 0), (148.0, 0), (168.0, 0), (498.0, 0), (718.0, 0)]",3000,2,617,47,8647,126,2022-10-23T01:21:21,true
6,122473,2022-09-30T12:00:00,2022-11-27T12:00:00,1003718051,设计,"[(39.0, 0), (39.0, 1), (79.0, 9)]",2800,2,98,13,670,10,2022-10-23T01:21:21,true
7,122618,2022-09-23T20:00:00,2022-11-20T23:59:00,1003452041,设计,"[(158.0, 37), (175.0, 2), (168.0, 10), (185.0, 0), (399.0, 3), (435.0, 2), (188.0, 21), (205.0, 0), (198.0, 34), (215.0, 3), (429.0, 14), (465.0, 3), (725.0, 2), (785.0, 5)]",30000,3,1891,63,34031,118,2022-10-23T01:21:21,true
8,123141,2022-10-17T19:00:00,2022-11-11T22:00:00,1003527584,出版,"[(28.0, 7), (58.0, 60), (88.0, 80), (98.0, 197), (128.0, 7), (198.0, 13), (376.0, 80), (386.0, 32)]",40000,5,754,78,74507,299,2022-10-23T01:21:21,true
9,123148,2022-10-14T19:00:00,2022-11-14T21:00:00,1003103318,设计,"[(288.0, 50), (288.0, 50), (298.0, 9), (298.0, 35)]",1000,2,668,33,41922,131,2022-10-23T01:21:21,true


## Compute average price
Next we parse the [(price,quantity purchased)] column to convert them into vectors of tuples.
After the conversion, compute the average price of the goods.
If the number of goods is more than two, remove the highest and lowest value first.
(This approach is flawed though,
since a project may have several expensive options tailored for few generous donors.
From this perspective, the average price should be weighted by the quantity purchased.
This however involves reverse causality.)

In [7]:
@chain main_info_df begin
    transform!(_,:price_quantity=>ByRow(parse_price_quantity_string);renamecols=false)
    transform!(_,:price_quantity=>ByRow(compute_avg_price)=>:avg_price)
end

Row,project_id,start_time,end_time,creator_id,category,price_quantity,goal,update_num,attention,comment_num,pledged,backer_num,scraped_time,status,avg_price
,String,DateTime,DateTime,String,String15,Array…,Float64,Float64,Float64,Float64,Float64,Float64,DateTime,Bool,Float64
1,122725,2022-10-12T18:00:00,2022-11-01T19:00:00,1001721653,桌游,"[(758, 9), (1088, 420), (198, 123), (168, 57), (25, 4), (45, 249)]",100000,16,4164,597,509117,423,2022-10-23T01:21:21,true,292
2,122816,2022-09-29T19:00:00,2022-11-22T23:59:00,1098393,出版,"[(119, 496), (129, 12), (189, 30), (328, 18), (388, 60)]",50000,2,1059,99,95207,584,2022-10-23T01:21:21,true,215
3,122810,2022-09-21T19:00:00,2022-11-01T18:16:00,1192788,出版,"[(678, 198), (698, 122), (249, 329), (478, 130), (88, 62), (500, 0)]",50000,11,1624,257,369063,714,2022-10-23T01:21:21,true,476
4,120778,2022-09-25T16:00:00,2022-11-06T12:00:00,1003655938,科技,"[(79, 30), (89, 10), (169, 3), (239, 22), (375, 0), (99, 12)]",20000,1,326,99,10213,53,2022-10-23T01:21:21,true,149
5,122864,2022-10-08T18:00:00,2022-11-08T18:00:00,1011894,桌游,"[(38, 65), (38, 10), (78, 1), (88, 0), (98, 46), (98, 0), (98, 0), (108, 10), (108, 1), (128, 0), (128, 0), (128, 0), (148, 0), (168, 0), (498, 0), (718, 0)]",3000,2,617,47,8647,126,2022-10-23T01:21:21,true,137
6,122473,2022-09-30T12:00:00,2022-11-27T12:00:00,1003718051,设计,"[(39, 0), (39, 1), (79, 9)]",2800,2,98,13,670,10,2022-10-23T01:21:21,true,39
7,122618,2022-09-23T20:00:00,2022-11-20T23:59:00,1003452041,设计,"[(158, 37), (175, 2), (168, 10), (185, 0), (399, 3), (435, 2), (188, 21), (205, 0), (198, 34), (215, 3), (429, 14), (465, 3), (725, 2), (785, 5)]",30000,3,1891,63,34031,118,2022-10-23T01:21:21,true,316
8,123141,2022-10-17T19:00:00,2022-11-11T22:00:00,1003527584,出版,"[(28, 7), (58, 60), (88, 80), (98, 197), (128, 7), (198, 13), (376, 80), (386, 32)]",40000,5,754,78,74507,299,2022-10-23T01:21:21,true,158
9,123148,2022-10-14T19:00:00,2022-11-14T21:00:00,1003103318,设计,"[(288, 50), (288, 50), (298, 9), (298, 35)]",1000,2,668,33,41922,131,2022-10-23T01:21:21,true,293


## Create discrete time index
we can divide the time passed from the commencement of a project by 12 hours
and take ceiling to get the time index.

In [8]:
@chain main_info_df begin
    groupby(_,:project_id)
    transform!(_,
    [:scraped_time,:start_time]=>((x,y)->compute_time_index(x,first(y);unit=Hour(12)))=>:time_index)
end

Row,project_id,start_time,end_time,creator_id,category,price_quantity,goal,update_num,attention,comment_num,pledged,backer_num,scraped_time,status,avg_price,time_index
,String,DateTime,DateTime,String,String15,Array…,Float64,Float64,Float64,Float64,Float64,Float64,DateTime,Bool,Float64,Int64
1,122725,2022-10-12T18:00:00,2022-11-01T19:00:00,1001721653,桌游,"[(758, 9), (1088, 420), (198, 123), (168, 57), (25, 4), (45, 249)]",100000,16,4164,597,509117,423,2022-10-23T01:21:21,true,292,21
2,122816,2022-09-29T19:00:00,2022-11-22T23:59:00,1098393,出版,"[(119, 496), (129, 12), (189, 30), (328, 18), (388, 60)]",50000,2,1059,99,95207,584,2022-10-23T01:21:21,true,215,47
3,122810,2022-09-21T19:00:00,2022-11-01T18:16:00,1192788,出版,"[(678, 198), (698, 122), (249, 329), (478, 130), (88, 62), (500, 0)]",50000,11,1624,257,369063,714,2022-10-23T01:21:21,true,476,63
4,120778,2022-09-25T16:00:00,2022-11-06T12:00:00,1003655938,科技,"[(79, 30), (89, 10), (169, 3), (239, 22), (375, 0), (99, 12)]",20000,1,326,99,10213,53,2022-10-23T01:21:21,true,149,55
5,122864,2022-10-08T18:00:00,2022-11-08T18:00:00,1011894,桌游,"[(38, 65), (38, 10), (78, 1), (88, 0), (98, 46), (98, 0), (98, 0), (108, 10), (108, 1), (128, 0), (128, 0), (128, 0), (148, 0), (168, 0), (498, 0), (718, 0)]",3000,2,617,47,8647,126,2022-10-23T01:21:21,true,137,29
6,122473,2022-09-30T12:00:00,2022-11-27T12:00:00,1003718051,设计,"[(39, 0), (39, 1), (79, 9)]",2800,2,98,13,670,10,2022-10-23T01:21:21,true,39,46
7,122618,2022-09-23T20:00:00,2022-11-20T23:59:00,1003452041,设计,"[(158, 37), (175, 2), (168, 10), (185, 0), (399, 3), (435, 2), (188, 21), (205, 0), (198, 34), (215, 3), (429, 14), (465, 3), (725, 2), (785, 5)]",30000,3,1891,63,34031,118,2022-10-23T01:21:21,true,316,59
8,123141,2022-10-17T19:00:00,2022-11-11T22:00:00,1003527584,出版,"[(28, 7), (58, 60), (88, 80), (98, 197), (128, 7), (198, 13), (376, 80), (386, 32)]",40000,5,754,78,74507,299,2022-10-23T01:21:21,true,158,11
9,123148,2022-10-14T19:00:00,2022-11-14T21:00:00,1003103318,设计,"[(288, 50), (288, 50), (298, 9), (298, 35)]",1000,2,668,33,41922,131,2022-10-23T01:21:21,true,293,17


check if there are duplicated time index

In [9]:
@chain begin
    nonunique(main_info_df,[:project_id,:time_index])
    main_info_df[_,:]
end

Row,project_id,start_time,end_time,creator_id,category,price_quantity,goal,update_num,attention,comment_num,pledged,backer_num,scraped_time,status,avg_price,time_index
,String,DateTime,DateTime,String,String15,Array…,Float64,Float64,Float64,Float64,Float64,Float64,DateTime,Bool,Float64,Int64
1,123125,2022-10-18T15:00:00,2022-12-15T15:00:00,1003179897,设计,"[(138, 100), (148, 79)]",20000,1,885,83,25524,169,2022-11-08T16:11:42,true,143,43
2,122573,2022-10-24T15:00:00,2022-12-13T15:00:00,1003936154,设计,"[(98, 21), (196, 11), (392, 11), (588, 13)]",60000,0,134,6,16350,21,2022-11-08T16:11:42,true,294,31
3,123099,2022-10-18T16:00:00,2022-11-18T12:00:00,1002518942,设计,"[(13, 62)]",500,2,93,15,815,47,2022-11-08T16:11:42,true,13,43
4,123053,2022-10-20T15:00:00,2022-11-25T22:00:00,1003201290,潮玩模型,"[(68, 308), (72, 134), (406, 52), (428, 8)]",30000,0,1510,114,57803,676,2022-11-08T16:11:42,true,239,39
5,123097,2022-10-13T15:00:00,2022-11-20T23:50:00,1001882141,设计,"[(89, 38), (89, 17), (89, 22)]",5000,1,181,45,6949,56,2022-11-08T16:11:42,true,89,53
6,123510,2022-11-04T15:00:00,2022-11-28T20:08:00,1002581709,设计,"[(328, 1), (388, 1)]",3000,0,69,4,716,2,2022-11-08T16:11:42,true,358,9
7,122986,2022-10-25T15:00:00,2022-11-30T15:00:00,1003699185,设计,"[(257, 5), (277, 0)]",5000,0,29,3,1323,8,2022-11-08T16:11:42,true,267,29
8,122814,2022-09-30T14:00:00,2022-10-31T23:59:00,1003026418,设计,"[(128, 35), (138, 0), (246, 16), (266, 0)]",20000,2,158,19,8429,51,2022-10-14T02:04:55,true,192,28
9,123010,2022-09-30T13:55:00,2022-10-31T23:59:00,1003191684,设计,"[(2880, 1), (2880, 1), (3080, 2), (3080, 0)]",2880,10,37,10,11922,5,2022-10-14T02:04:55,true,2980,28


It turns out that for some projects have multiple observations
within 12 hours. For those data, we only keep
the latest observations in a 12-hour window.

In [10]:
@chain main_info_df begin
    sort!(_,:scraped_time)
    unique!(_,[:project_id,:time_index];keep=:last)
end

Row,project_id,start_time,end_time,creator_id,category,price_quantity,goal,update_num,attention,comment_num,pledged,backer_num,scraped_time,status,avg_price,time_index
,String,DateTime,DateTime,String,String15,Array…,Float64,Float64,Float64,Float64,Float64,Float64,DateTime,Bool,Float64,Int64
1,122885,2022-09-28T12:00:00,2022-10-13T21:00:00,3571057,桌游,"[(98, 400), (108, 16), (185, 88), (12, 58)]",30000,2,1166,94,57924,496,2022-10-11T02:14:44,true,103,26
2,122035,2022-09-17T12:00:00,2022-10-31T12:00:00,1427792,游戏,"[(24, 236), (66, 8), (88, 86), (148, 27), (248, 8), (666, 28)]",30000,7,1155,478,45200,362,2022-10-11T02:14:44,true,138,48
3,122933,2022-10-09T19:00:00,2022-11-14T19:00:00,1002795157,出版,"[(88, 94), (98, 0), (118, 200), (128, 22)]",40000,0,969,90,34462,274,2022-10-11T02:14:44,true,108,3
4,122894,2022-09-28T19:00:00,2022-11-16T19:00:00,1000806663,出版,"[(299, 7), (388, 26), (398, 0), (3660, 1), (3680, 0), (3990, 14)]",50000,2,695,61,71511,43,2022-10-11T02:14:44,true,2032,25
5,122734,2022-09-20T17:00:00,2022-10-19T23:59:00,1001797063,潮玩模型,"[(189, 21), (520, 78), (599, 2), (599, 9), (520, 28), (169, 24), (169, 11), (169, 11), (169, 3), (69, 5)]",50000,2,848,109,74349,147,2022-10-11T02:14:44,true,313,41
6,122476,2022-09-13T12:01:00,2022-10-28T12:01:00,5376764,桌游,"[(99, 227), (109, 42), (209, 1), (289, 1), (268, 13), (268, 1), (699, 0)]",20000,3,613,67,30383,277,2022-10-11T02:14:44,true,229,56
7,122450,2022-09-04T12:00:00,2022-11-03T12:00:00,1003921284,潮玩模型,"[(59, 3), (66, 0), (66, 0), (66, 0), (66, 0), (66, 0), (66, 0), (66, 0), (66, 0), (66, 0) … (66, 0), (66, 0), (66, 1), (89, 0), (177, 0), (198, 0), (354, 1), (767, 0), (267, 0), (534, 0)]",5000,1,65,36,597,5,2022-10-11T02:14:44,true,130,74
8,122618,2022-09-23T20:00:00,2022-11-20T23:59:00,1003452041,设计,"[(158, 33), (175, 2), (168, 12), (185, 0), (399, 6), (435, 2), (188, 19), (205, 0), (198, 28), (215, 3), (429, 15), (465, 3), (725, 2), (785, 5)]",30000,1,1833,59,33797,110,2022-10-11T02:14:44,true,316,35
9,122901,2022-10-03T09:03:00,2022-11-09T12:03:00,7222657,桌游,"[(79, 0), (79, 30), (89, 58)]",10000,1,368,73,7563,85,2022-10-11T02:14:44,true,79,16


## Filter projects

In [11]:
# Only keep projects whose life cycle is fully covered
@chain main_info_df begin
    sort!(_,:time_index)
    groupby(_,:project_id)
    subset!(_,
    :time_index=>x->minimum(x)==1,
    [:end_time,:scraped_time]=>((x,y)->last(x)<=last(y)))
end

Row,project_id,start_time,end_time,creator_id,category,price_quantity,goal,update_num,attention,comment_num,pledged,backer_num,scraped_time,status,avg_price,time_index
,String,DateTime,DateTime,String,String15,Array…,Float64,Float64,Float64,Float64,Float64,Float64,DateTime,Bool,Float64,Int64
1,122558,2022-10-11T11:00:00,2022-11-20T12:00:00,1003647765,游戏,"[(18, 16), (32, 11), (68, 3), (168, 3), (648, 0), (888, 0), (1588, 0)]",10000,0,282,35,1363,34,2022-10-11T14:14:44,true,361,1
2,123023,2022-10-11T17:00:00,2022-10-31T12:00:00,1002988064,设计,"[(128, 2), (148, 0)]",5000,0,2,2,256,2,2022-10-12T01:55:31,true,138,1
3,122854,2022-10-11T19:00:00,2022-11-10T22:00:00,7081694,潮玩模型,"[(315, 4), (340, 2), (308, 1), (192, 2), (827, 26), (2200, 8), (2242, 0), (420, 20), (427, 4), (239, 5), (240, 646), (2923, 2), (805, 50), (2823, 20)]",50000,2,977,112,310761,141,2022-10-12T01:55:31,true,932,1
4,122746,2022-10-11T19:00:00,2022-10-29T19:00:00,4786218,潮玩模型,"[(229, 28), (90, 11), (460, 8), (460, 6), (460, 5), (690, 2), (3550, 226), (2550, 16), (1020, 3), (666, 151), (1200, 307), (860, 10), (980, 124), (299, 11), (975, 1), (650, 1), (650, 1), (650, 1), (235, 2), (95, 0)]",200000,18,1787,79,1470026,331,2022-10-12T01:55:31,true,729,1
5,123091,2022-10-11T19:00:00,2022-10-25T19:00:00,1001348035,出版,"[(488, 2), (468, 13), (298, 2), (138, 11), (188, 1), (238, 0), (228, 17), (178, 1), (168, 3)]",10000,1,208,31,13920,48,2022-10-12T01:55:31,true,252,1
6,123123,2022-10-12T19:00:00,2022-11-12T19:00:00,7170148,出版,"[(288, 123), (352, 9), (416, 50), (426, 1)]",20000,1,961,64,59818,166,2022-10-13T02:00:58,true,384,1
7,123114,2022-10-12T19:00:00,2022-10-24T19:00:00,1000371586,出版,"[(308, 50), (328, 100), (338, 75), (338, 29), (348, 3), (378, 50), (388, 33)]",30000,4,935,124,116120,238,2022-10-13T02:00:58,true,346,1
8,122725,2022-10-12T18:00:00,2022-10-28T19:00:00,1001721653,桌游,"[(758, 11), (1088, 304), (198, 85), (25, 3), (45, 165)]",100000,5,4095,467,363470,313,2022-10-13T02:00:58,true,334,1
9,123103,2022-10-12T20:00:00,2022-11-12T20:00:00,1001651349,潮玩模型,"[(128, 50), (128, 20), (158, 50), (158, 39), (278, 31), (45, 0)]",10000,3,877,88,31652,184,2022-10-13T02:00:58,true,143,1


In [12]:
# Check if the project end date is modified halfway
@chain main_info_df begin
    groupby(_,:project_id)
    subset(_,:end_time=> x-> maximum(x) != minimum(x))
end

Row,project_id,start_time,end_time,creator_id,category,price_quantity,goal,update_num,attention,comment_num,pledged,backer_num,scraped_time,status,avg_price,time_index
,String,DateTime,DateTime,String,String15,Array…,Float64,Float64,Float64,Float64,Float64,Float64,DateTime,Bool,Float64,Int64
1,122854,2022-10-11T19:00:00,2022-11-10T22:00:00,7081694,潮玩模型,"[(315, 4), (340, 2), (308, 1), (192, 2), (827, 26), (2200, 8), (2242, 0), (420, 20), (427, 4), (239, 5), (240, 646), (2923, 2), (805, 50), (2823, 20)]",50000,2,977,112,310761,141,2022-10-12T01:55:31,true,932,1
2,123123,2022-10-12T19:00:00,2022-11-12T19:00:00,7170148,出版,"[(288, 123), (352, 9), (416, 50), (426, 1)]",20000,1,961,64,59818,166,2022-10-13T02:00:58,true,384,1
3,122725,2022-10-12T18:00:00,2022-10-28T19:00:00,1001721653,桌游,"[(758, 11), (1088, 304), (198, 85), (25, 3), (45, 165)]",100000,5,4095,467,363470,313,2022-10-13T02:00:58,true,334,1
4,123172,2022-10-16T19:00:00,2022-11-20T21:00:00,1002374318,设计,"[(288, 1), (298, 0)]",5000,0,80,19,288,1,2022-10-17T01:13:07,true,293,1
5,123087,2022-10-16T20:00:00,2022-11-22T23:59:00,1668553,潮玩模型,"[(299, 5), (349, 6), (628, 12)]",6000,0,234,14,11185,28,2022-10-17T01:13:07,true,349,1
6,122965,2022-10-16T19:00:00,2022-10-31T19:00:00,1002465033,设计,"[(899, 0), (899, 1), (899, 1), (899, 0), (899, 0), (899, 0), (899, 0), (899, 0), (899, 0), (899, 1)]",1000,0,248,12,2697,3,2022-10-17T01:13:07,true,899,1
7,122234,2022-10-19T19:00:00,2022-11-18T23:59:00,1004110743,潮玩模型,"[(468, 10), (478, 10), (488, 20), (498, 41), (926, 5), (966, 10)]",30000,2,985,32,53965,77,2022-10-20T01:11:11,true,598,1
8,123182,2022-10-20T19:00:00,2022-11-20T19:00:00,7571228,出版,"[(288, 500), (288, 1000), (198, 5866), (198, 1000), (768, 450)]",100000,1,8812,2070,2138451,4951,2022-10-21T01:24:05,true,258,1
9,122541,2022-10-22T12:00:00,2022-11-22T12:00:00,1003244201,设计,"[(398, 0), (398, 0), (788, 0), (428, 0), (428, 0), (818, 0)]",1000,0,4,8,0,0,2022-10-22T13:22:33,true,510,1


In [13]:
# Remove these projects
@chain main_info_df begin
    groupby(_,:project_id)
    subset!(_,:end_time=> x-> maximum(x) == minimum(x))
end

Row,project_id,start_time,end_time,creator_id,category,price_quantity,goal,update_num,attention,comment_num,pledged,backer_num,scraped_time,status,avg_price,time_index
,String,DateTime,DateTime,String,String15,Array…,Float64,Float64,Float64,Float64,Float64,Float64,DateTime,Bool,Float64,Int64
1,122558,2022-10-11T11:00:00,2022-11-20T12:00:00,1003647765,游戏,"[(18, 16), (32, 11), (68, 3), (168, 3), (648, 0), (888, 0), (1588, 0)]",10000,0,282,35,1363,34,2022-10-11T14:14:44,true,361,1
2,123023,2022-10-11T17:00:00,2022-10-31T12:00:00,1002988064,设计,"[(128, 2), (148, 0)]",5000,0,2,2,256,2,2022-10-12T01:55:31,true,138,1
3,122746,2022-10-11T19:00:00,2022-10-29T19:00:00,4786218,潮玩模型,"[(229, 28), (90, 11), (460, 8), (460, 6), (460, 5), (690, 2), (3550, 226), (2550, 16), (1020, 3), (666, 151), (1200, 307), (860, 10), (980, 124), (299, 11), (975, 1), (650, 1), (650, 1), (650, 1), (235, 2), (95, 0)]",200000,18,1787,79,1470026,331,2022-10-12T01:55:31,true,729,1
4,123091,2022-10-11T19:00:00,2022-10-25T19:00:00,1001348035,出版,"[(488, 2), (468, 13), (298, 2), (138, 11), (188, 1), (238, 0), (228, 17), (178, 1), (168, 3)]",10000,1,208,31,13920,48,2022-10-12T01:55:31,true,252,1
5,123114,2022-10-12T19:00:00,2022-10-24T19:00:00,1000371586,出版,"[(308, 50), (328, 100), (338, 75), (338, 29), (348, 3), (378, 50), (388, 33)]",30000,4,935,124,116120,238,2022-10-13T02:00:58,true,346,1
6,123103,2022-10-12T20:00:00,2022-11-12T20:00:00,1001651349,潮玩模型,"[(128, 50), (128, 20), (158, 50), (158, 39), (278, 31), (45, 0)]",10000,3,877,88,31652,184,2022-10-13T02:00:58,true,143,1
7,123013,2022-10-13T19:00:00,2022-11-13T22:00:00,1003987233,设计,"[(569, 37), (579, 0), (1099, 8), (1150, 24)]",10000,1,514,59,57445,47,2022-10-14T02:04:55,true,839,1
8,122815,2022-10-13T19:00:00,2022-11-13T19:00:00,1002762410,游戏,"[(68, 64), (98, 33), (188, 44), (218, 30), (488, 41), (1528, 25)]",20000,5,1708,166,81077,240,2022-10-14T02:04:55,true,248,1
9,121473,2022-10-13T20:00:00,2022-11-17T20:00:00,1001769215,桌游,"[(168, 300), (178, 133), (368, 300), (378, 49)]",100000,5,2228,277,202996,707,2022-10-14T02:04:55,true,273,1


Interpolate missing values

In [14]:
# complete data for each project by time index
main_info_df = @chain main_info_df begin
    groupby(_,:project_id)
    complete(_,:time_index)
end

Row,project_id,start_time,end_time,creator_id,category,price_quantity,goal,update_num,attention,comment_num,pledged,backer_num,scraped_time,status,avg_price,time_index
,String,DateTime?,DateTime?,String?,String15?,Array…?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,DateTime?,Bool?,Float64?,Int64
1,122558,2022-10-11T11:00:00,2022-11-20T12:00:00,1003647765,游戏,"[(18, 16), (32, 11), (68, 3), (168, 3), (648, 0), (888, 0), (1588, 0)]",10000,0,282,35,1363,34,2022-10-11T14:14:44,true,361,1
2,122558,2022-10-11T11:00:00,2022-11-20T12:00:00,1003647765,游戏,"[(18, 33), (32, 15), (68, 6), (168, 3), (648, 0), (888, 0), (1588, 0)]",10000,0,293,35,2020,58,2022-10-12T01:55:31,true,361,2
3,122558,2022-10-11T11:00:00,2022-11-20T12:00:00,1003647765,游戏,"[(18, 38), (32, 15), (68, 6), (168, 3), (648, 0), (888, 0), (1588, 0)]",10000,0,297,35,2110,63,2022-10-12T14:04:50,true,361,3
4,122558,2022-10-11T11:00:00,2022-11-20T12:00:00,1003647765,游戏,"[(18, 43), (32, 15), (68, 6), (168, 3), (648, 0), (888, 0), (1588, 0)]",10000,0,327,35,2200,68,2022-10-13T02:00:58,true,361,4
5,122558,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,5
6,122558,2022-10-11T11:00:00,2022-11-20T12:00:00,1003647765,游戏,"[(18, 45), (32, 14), (68, 8), (168, 4), (648, 0), (888, 0), (1588, 0)]",10000,0,338,35,2540,72,2022-10-14T02:04:55,true,361,6
7,122558,2022-10-11T11:00:00,2022-11-20T12:00:00,1003647765,游戏,"[(18, 46), (32, 15), (68, 8), (168, 4), (648, 0), (888, 0), (1588, 0)]",10000,0,338,36,2590,74,2022-10-14T13:17:11,true,361,7
8,122558,2022-10-11T11:00:00,2022-11-20T12:00:00,1003647765,游戏,"[(18, 52), (32, 16), (68, 9), (168, 4), (648, 0), (888, 0), (1588, 0)]",10000,0,350,40,2798,82,2022-10-15T01:15:35,true,361,8
9,122558,2022-10-11T11:00:00,2022-11-20T12:00:00,1003647765,游戏,"[(18, 52), (32, 16), (68, 9), (168, 4), (648, 0), (888, 0), (1588, 0)]",10000,0,350,40,2798,82,2022-10-15T13:18:07,true,361,9


In [15]:
# forward fill the time-invariant variables
# interpolate the time-varying variables
@chain main_info_df begin
    groupby(_,:project_id)
    transform!(_,
    [:status,:start_time,:end_time,:creator_id,
    :avg_price,:category,:price_quantity,:goal].=>x->fill_missing(x;method=:ffill),
    [:pledged,:backer_num,:update_num,
    :attention,:comment_num].=>interpolate_missing_linear;
    renamecols=false)
    select!(_,Not([:scraped_time]))
end

Row,project_id,start_time,end_time,creator_id,category,price_quantity,goal,update_num,attention,comment_num,pledged,backer_num,status,avg_price,time_index
,String,DateTime?,DateTime?,String?,String15?,Array…?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Bool?,Float64?,Int64
1,122558,2022-10-11T11:00:00,2022-11-20T12:00:00,1003647765,游戏,"[(18, 16), (32, 11), (68, 3), (168, 3), (648, 0), (888, 0), (1588, 0)]",10000,0,282,35,1363,34,true,361,1
2,122558,2022-10-11T11:00:00,2022-11-20T12:00:00,1003647765,游戏,"[(18, 33), (32, 15), (68, 6), (168, 3), (648, 0), (888, 0), (1588, 0)]",10000,0,293,35,2020,58,true,361,2
3,122558,2022-10-11T11:00:00,2022-11-20T12:00:00,1003647765,游戏,"[(18, 38), (32, 15), (68, 6), (168, 3), (648, 0), (888, 0), (1588, 0)]",10000,0,297,35,2110,63,true,361,3
4,122558,2022-10-11T11:00:00,2022-11-20T12:00:00,1003647765,游戏,"[(18, 43), (32, 15), (68, 6), (168, 3), (648, 0), (888, 0), (1588, 0)]",10000,0,327,35,2200,68,true,361,4
5,122558,2022-10-11T11:00:00,2022-11-20T12:00:00,1003647765,游戏,"[(18, 43), (32, 15), (68, 6), (168, 3), (648, 0), (888, 0), (1588, 0)]",10000,0,332,35,2370,70,true,361,5
6,122558,2022-10-11T11:00:00,2022-11-20T12:00:00,1003647765,游戏,"[(18, 45), (32, 14), (68, 8), (168, 4), (648, 0), (888, 0), (1588, 0)]",10000,0,338,35,2540,72,true,361,6
7,122558,2022-10-11T11:00:00,2022-11-20T12:00:00,1003647765,游戏,"[(18, 46), (32, 15), (68, 8), (168, 4), (648, 0), (888, 0), (1588, 0)]",10000,0,338,36,2590,74,true,361,7
8,122558,2022-10-11T11:00:00,2022-11-20T12:00:00,1003647765,游戏,"[(18, 52), (32, 16), (68, 9), (168, 4), (648, 0), (888, 0), (1588, 0)]",10000,0,350,40,2798,82,true,361,8
9,122558,2022-10-11T11:00:00,2022-11-20T12:00:00,1003647765,游戏,"[(18, 52), (32, 16), (68, 9), (168, 4), (648, 0), (888, 0), (1588, 0)]",10000,0,350,40,2798,82,true,361,9


In [16]:
# save the cleaned data
# save_object("$(project_dir)/data/prev_data/edited_data/main_info_df.jld2",main_info_df)